In [11]:
quran_file = 'nlp-exploring-datasets/religious_text/id_text_with_orthographies.txt'

In [12]:
with open(quran_file) as fp:
    temp = fp.readlines()

In [13]:
verses = {}

for verse in temp:
    spls = verse.split('\t')
    verse_no = spls[0]
    verse_con = spls[1].strip()
    verses[verse_no] = verse_con

In [1]:
import json

with open('analysis.json') as fp:
    analysis_data = json.load(fp)

with open('quran.json') as fp:
    verses = json.load(fp)

In [2]:
import csv

vavs = {}

with open('Vav.csv') as fp:
    reader = csv.reader(fp, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        else:
            vavs[row[0]] = int(row[1])
vavs_keys = set(vavs.keys())


In [15]:
vavs['والارض']

0

In [34]:
import re

verses_prep = {}
for verse_id, text in verses.items():
    text = re.sub(r'[' + analysis_data['pun'] + 'ّ]', '', text)
    text = re.sub(r'[\s]+', ' ', text)
    for key, val in analysis_data['repl'].items():
        text = re.sub(r'[' + key + ']', val, text)
    verse_tokens = [w for w in text.split(' ')]
    commons = set.intersection(set(verse_tokens), vavs_keys)
    if commons:
        verse_tokens_new = []
        for i, w in enumerate(verse_tokens):
            if w in commons:
                if vavs[w] == 2:
                    verse_tokens_new.append(str(w[0]))
                    verse_tokens_new.append(w[1:])
                elif vavs[w] == 0:
                    if w == 'وال':
                        if verse_tokens[i - 1] == 'من':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'ورد':
                        if verse_tokens[i + 1] == 'ماء':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وكل':
                        if verse_tokens[i + 1] == 'بكم':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'ولي':
                        if verse_tokens[i + 1] == 'فيها' or verse_tokens[i + 1] == 'نعجة' or verse_tokens[i + 1] == 'دين':
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                        else:
                            verse_tokens_new.append(w)
                    elif w == 'وليتم':
                        if verse_tokens[i + 1] == 'مدبرين':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وليقولوا':
                        if verse_tokens[i + 1] == 'قولا':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وهب':
                        if verse_tokens[i - 1] == 'الذي':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وفي':
                        if verse_tokens[i - 1] == 'الذي':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والارض':
                        if i == 0 and verse_tokens[i + 1] in ['ذات', 'و']:
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والسماء':
                        if i == 0 and verse_tokens[i + 1] in ['ذات', 'و']:
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والشمس':
                        if i == 0 and verse_tokens[i + 1] == 'و':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والقران':
                        if verse_tokens[i + 1] in ['الحكيم', 'ذي', 'المجيد']:
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والقمر':
                        if (i > 0 and verse_tokens[i - 1] == 'كلا') or (i == 0 and verse_tokens[i + 1] == 'اذا'):
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والكتاب':
                        if verse_tokens[i + 1] == 'المبين':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والنهار':
                        if i == 0 and verse_tokens[i + 1] == 'اذا':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وهذا':
                        if i == 0 and verse_tokens[i + 1] == 'البلد':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وربك':
                        if i > 0 and verse_tokens[i - 1] == 'فلا':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'والله':
                        if i > 0 and verse_tokens[i + 1] == 'ربنا':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    elif w == 'وربنا':
                        if i > 0 and verse_tokens[i - 1] == 'بلي':
                            verse_tokens_new.append(w)
                        else:
                            verse_tokens_new.append(str(w[0]))
                            verse_tokens_new.append(w[1:])
                    else:
                        verse_tokens_new.append(w)
                elif vavs[w] == 1:
                    verse_tokens_new.append(w)
            else:
                verse_tokens_new.append(w)
        verses_prep[verse_id] = verse_tokens_new
    else:
        verses_prep[verse_id] = verse_tokens
quran_text = ' '.join([' '.join(val) for val in verses_prep.values()])
words = quran_text.split(' ')
chars = sorted([ch for ch in set(list(quran_text)) if ch != ' '])

# words_y = [(words[i - 1], words[i], words[i + 1]) for i, w in enumerate(words) if 'أ' in w]

In [38]:
words_set = sorted(set(words))
words_set[0] = '[UNK]'
words_map = {w:i for i, w in enumerate(words_set)}
with open('word_codes.json', 'w') as fw:
    json.dump(words_map, fw, ensure_ascii=False)

with open('words.txt', 'w') as fw:
    for w in words_set:
        fw.write(w + '\n')

In [37]:
with open('quran_prep.json', 'w') as fw:
    json.dump(verses_prep, fw, ensure_ascii=False)

In [3]:
words_sub = set([w for w in words if w and w[0] == 'و'])
words_sub_sub = set([w for w in words_sub if w[1:] not in words])

In [5]:
with open('words_starts_with_v.txt', 'w') as fw:
    for w in words_sub:
        fw.write(w + '\n')

In [ ]:
words_set = set([w for w in words if w[0] != 'و'])
words_one = [w for w in words_set if len(w) == 1]

In [39]:
index = {}
for verse_id, verse_seq in verses_prep.items():
    for word_id in [words_map[w] for w in verse_seq if w]:
        if not word_id in index:
            index[word_id] = set()
        index[word_id].add(verse_id)

In [40]:
for key in index:
    index[key] = sorted(list(index[key]))

with open('verse_index.json', 'w') as fw:
    json.dump(index, fw)

In [1]:
x = {1, 2, 3}
y = [2, 5, 7, 9]
a = [x, y]
set.intersection(*a)

{2}

In [43]:
[(words[i - 3], words[i - 2], words[i - 1], words[i], words[i + 1]) for i in range(len(words)) if words[i] == 'والله']

[('الا', 'ان', 'قالوا', 'والله', 'ربنا')]